***번개장터 데이터 가져오기***

In [57]:
def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df


sample_df = concat_bungae_files()
sample_df


/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/2464143370.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/2464143370.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/2464143370.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')


,product_id,product_name,cat_id
0,201926367,여성야상패딩,310090050
1,173779753,여성파라점퍼스XL,310090050
2,213485472,경량 패딩 새상품 가니 점퍼 스타일,310090050
3,219480633,"휠라 롱패딩,아디다스 패딩, 양털 점퍼, 코트 일괄",310090050
4,202566238,무료배송)날씬해보이는 거위솜털 여성패딩점퍼55,310090050
...,...,...,...
1391171,164097143,전통매듭목걸이/목걸이/전통목걸이/쥬얼리/매듭,400999
1391172,174926618,인조모 가발 붙임머리 긴머리,400999
1391173,219603119,마리떼 헤어클립 구해요,400999
1391174,224250961,새제품)호피 헤어밴드,400999


# 베이스 코드

In [61]:
# -- 예시로 여성 패딩 카테고리만 가져와보기 --
# 310090050 롱패딩
# 310090060 숏패딩
# 310090020 블루종/항공점퍼
df = sample_df[sample_df['cat_id'].isin(['310090050', '310090060', '310090020'])].copy()

In [65]:
# -- 전처리 --
# -- 기호 없애기 --
text = 'bcbg 정품 오리털패딩(택포)'
import string
def remove_punct(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

df['product_name'] = df['product_name'].apply(lambda x : remove_punct(x))

In [72]:
df = df.reset_index(drop=True)

***Word2Vec, FastText 활용하지 않기***

In [67]:
"""활용 X"""
# from gensim.models import Word2Vec
# from gensim.models.fasttext import FastText
# from konlpy.tag import Mecab
# import gensim

**숏패딩**

In [77]:
short = df[df['cat_id'] == '310090060'].copy()
short = short.reset_index(drop=True).copy()
short.head()

,product_id,product_name,cat_id
0,215721614,새상품코오롱스포츠 구스다운택그대로있음초특가,310090060
1,205175946,코오롱 여자패딩,310090060
2,221433807,언더아머경량패딩,310090060
3,211406321,노스페이스 눕시 크롭 스톰 블루 s,310090060
4,210976721,새상품 몽클헤어 패딩자켓,310090060


In [89]:
print(len(short))

9990


In [120]:
short_filtered = short[~short['product_name'].str.contains('롱|항공|블루종')].copy()
short_filtered = short_filtered['product_name'][short_filtered['product_name'].str.contains('숏')].copy()
print(len(short_filtered))
short_filtered.head(10)

2375


7               가격 내림타미힐피거 아동 키즈 숏패딩
9                     웰론 100 시보리 숏패딩
12                    미니마리 숏패딩 프리사이즈
22              비뮤즈맨션 라운드 카라 숏패딩 덕다운
30     아뜨랑스 자체제작 몽블린 폭스퍼 a라인 덕다운 숏패딩
31                            골덴 숏패딩
35                          지오다노 숏패딩
40                         lap 랩 숏패딩
49    meeting cici 볼드코듀로이 숏 패딩 핑크 우먼
51                           앤드지 숏패딩
Name: product_name, dtype: object

**롱패딩**

In [96]:
long = df[df['cat_id'] == '310090050'].copy()
long = long.reset_index(drop=True).copy()
long.head()

,product_id,product_name,cat_id
0,201926367,여성야상패딩,310090050
1,173779753,여성파라점퍼스xl,310090050
2,213485472,경량 패딩 새상품 가니 점퍼 스타일,310090050
3,219480633,휠라 롱패딩아디다스 패딩 양털 점퍼 코트 일괄,310090050
4,202566238,무료배송날씬해보이는 거위솜털 여성패딩점퍼55,310090050


In [118]:
long_filtered = long[~long['product_name'].str.contains('숏|항공|블루종')].copy()
long_filtered = long_filtered['product_name'][long_filtered['product_name'].str.contains('롱')].copy()
print(len(long_filtered))
long_filtered.head(20)

4545


3               휠라 롱패딩아디다스 패딩 양털 점퍼 코트 일괄
5                              데상트 여성 롱패딩
7                                코오롱 다운자켓
10    하몽마켓 남여공용 야상 롱 패딩 long padding coat
11                      노스페이스 경량 롱 거위털 여성
12                               노스페이스롱패딩
13                    새상품 여성 블랙 롱패딩 55 급처
17                 xs파타고니아 우먼스 경량 덕다운 롱패딩
18                           코오롱안타티카구스 다운
19        여성 간절기 패딩 코오롱 잭니클라우스 lwuas21021
20                               스파이더 롱패딩
24                             나인식스뉴욕 롱패딩
26        밀레 여성용흰색화이트컬러후드다운롱패딩점퍼85사이즈호구제샵
27                        콜롬비아 롱패딩s 44 추천
28                            하이드아웃 롱 플리스
31                           코오롱 체크무늬 롱패딩
32              soup 숲 연베이지 덕다운 패딩 롱점퍼 55
36                         원가49 디스커버리 롱패딩
40      빈폴 bbench 덕다운 벤치다운 롱패딩 bo8738e01t
41                               아디다스 롱패딩
Name: product_name, dtype: object

**항공점퍼|블루종**

In [122]:
hangong = df[df['cat_id'] == '310090020'].copy()
hangong = hangong.reset_index(drop=True).copy()
hangong.head()

,product_id,product_name,cat_id
0,222470767,럭키슈에뜨 트위드자켓,310090020
1,221991996,낫띵리튼 블루종 marche summer blouson ecru,310090020
2,104222621,봄 가을 점퍼,310090020
3,225210436,보헤미안서울 리벳 포인트 밴딩 점퍼,310090020
4,214800133,슈프림 스키틀즈 미첼 네스 콜라보 바시티 재킷 퍼플새상품,310090020


In [123]:
hangong_filtered = hangong[~hangong['product_name'].str.contains('숏|롱')].copy()
hangong_filtered = hangong_filtered['product_name'][hangong_filtered['product_name'].str.contains('항공|블루종')].copy()
print(len(hangong_filtered))
hangong_filtered.head(20)

2842


1          낫띵리튼 블루종 marche summer blouson ecru
7               더언더블부트아케 스웨이드 테디 바시티 블루종 자켓 블랙
8                            지컷 gcut 와펜 블루종 점퍼
14                     어텀 윈터 나탈리점퍼 블루종점퍼 어썸 닻별
16                            라퍼지포우먼 에센셜 블루종 m
18                               크롭 셔링 항공점퍼 블랙
20                  밀리언코르 투웨이 미니멀 카라 스냅 블루종 점퍼
25                            빈티지크롭라코스테 크롭 블루종
30                            인디브랜드 니트 블루종 가디건
36                                 유니클로블루종 여자l
37                                카이아크만 항공 패딩s
39                     포저 레글런 블루종 자켓 오프 화이트 색상
41    mlb 바시티자켓 하이틴 야구잠바 뉴욕양키즈 항공점퍼 아우터y2k 빈티지
46                      워크 블루종 바시티 점퍼 에이블리 더블콩
47                    완전 새상품콜릿 마일드레더 항공 플라켓 점퍼
50                                  인더클로젯 항공점퍼
55                            코페르니 블루종 후디 자켓 s
58                   슬로우앤드 베어 썸머블루종 와샤점퍼 파스텔민트
61            luisa cerano 블루종자켓 사이즈66 운포3만5천원
66                            커버낫 골덴 항공점퍼s남여공용
Name: product_name, dtype: object

In [ ]:
"""-- 여기까지 여성의류 > 패딩 > 숏패딩, 롱패딩, 항공점퍼|블루종 예시 --"""

# 전체 카테고리에 대한 코드

In [588]:
def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df

# -- 전처리 --
# -- 기호 없애기 --
import string
def remove_punct(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

# -- category_df -- 
# def bring_fashion_cat_csv():
#     """
#     번개장터 카테고리 id와 카테고리 이름이 들어있는
#     파일 가져오는 함수
#     """
#     final_cat = pd.read_csv('./final_category.csv')
#     final_cat = final_cat.drop('Unnamed: 0', axis=1).copy()
#     final_cat['cat_id'] = final_cat['cat_id'].astype(str)
    
#     return final_cat

# def merge_two_df():
#     # -- 번개장터 전체 데이터 --
#     sample_df = concat_bungae_files()
#     sample_df['product_name'] = sample_df['product_name'].apply(lambda x : remove_punct(x))
#     # -- category 데이터프레임 --
#     category_df = bring_fashion_cat_csv()
#     category_df = category_df.fillna('no_cat')
#     # -- 둘이 합치기 --
#     df = sample_df.merge(category_df, on='cat_id', how='left')
#     return df

import json
from collections import defaultdict
            
def bring_fashion_cat_csv():
    """
    번개장터 카테고리 id와 카테고리 이름이 들어있는
    파일 가져오는 함수
    """
    final_cat = pd.read_csv('./final_category.csv')
    final_cat = final_cat.drop('Unnamed: 0', axis=1).copy()
    final_cat['cat_id'] = final_cat['cat_id'].astype(str)
    
    return final_cat

def get_cat_name(cat_id:str) -> str:
    """
    카테고리 id를 입력하면 카테고리 이름을 반환하는 함수 => 중분류에 대해서
    """
    final_cat = bring_fashion_cat_csv()
    if len(cat_id) == 6: # 중분류
        return list(final_cat[final_cat['cat_id'] == cat_id]['cat2'])[0]
    elif len(cat_id) == 9: # 대분류
        return list(final_cat[final_cat['cat_id'] == cat_id]['cat3'])[0]

# -- subs 가 있는 것들만 dictionary로 뽑아내기 --
def make_category_dict_with_subs() -> dict:
    
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)
        
    category_dict_with_subs = dict()
    for main, mids in data.items():
        for mid, subs in mids.items():
            d = defaultdict()
            if subs != [None]: # 하위 sub들이 있을 때 
                d[mid] = subs
            category_dict_with_subs.update(d)
    return category_dict_with_subs


# -- 중분류 내에서의 소분류 morphs 키워드 가져오는 함수 --
def make_subcat_morphs_by_midcat(cat_dict:dict) -> dict:
    """
    make_category_dict_with_subs 함수로 부터 받은 딕션어리로
    midcat의 하위 subcat들의 morphs 키워드 가져오는 함수 
    """
    mecab = Mecab()
    d = defaultdict(list)

    for mid, subs in cat_dict.items():
        tmp_dict = dict()
        for sub in subs:

            mid_morph = mecab.morphs(get_cat_name(mid))
            sub_morph = mecab.morphs(remove_punct(get_cat_name(sub)))
            unique_sub_morph = set(sub_morph) - set(mid_morph)
            unique_sub_morph = list(unique_sub_morph)
            if unique_sub_morph == []:
                d[mid] += sub_morph
            d[mid] += unique_sub_morph

    return dict(d)

# -- 위의 두 함수 합치기 --
def get_dict_from_midcat_with_subs():
    cat_dict = make_category_dict_with_subs()
    morphs_dict = make_subcat_morphs_by_midcat(cat_dict)
    return morphs_dict


# -- 다음으로는 mid만 있는 것들만 뽑아내기 --
def make_category_lists_without_subs():
    """
    소분류 카테고리가 없는 중분류 리스트 => cat_ids_without_subs
    """
    # -- 패션 cat_id 들어있는 json 파일 가져오기 --
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)

    cat_list_without_subs = list()
    morphs_list = list()
    for main, mids in data.items():
        for mid in mids:
            if mids[mid] == [None]:
                morphs_list += mecab.morphs(remove_punct(get_cat_name(mid)))
                cat_list_without_subs.append(mid)
    
    morphs_list = list(set(morphs_list))
    
    return morphs_list, cat_list_without_subs

In [643]:
# -- 카테고리 번호가 9인, 즉 sub 카테고리가 있는 상품들에 대해 
def make_df_for_nine_nums_cat_ids():
    sample_df = concat_bungae_files()
    morphs_dict_with_subs = get_dict_from_midcat_with_subs()
    
    final_df = pd.DataFrame()
    for cat_id in list(sample_df['cat_id'].unique()):
        if len(cat_id) == 9:
            try:
                tmp_df = sample_df[sample_df['cat_id'] == cat_id]
                cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))
                mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
                midcat_morphs = morphs_dict_with_subs[cat_id[:6]]

                cat_id_morphs = [morph for morph in cat_id_morphs if morph in midcat_morphs]
                other_morphs = [morph for morph in midcat_morphs if morph not in cat_id_morphs]
                opposite_keywords = '|'.join(other_morphs)

                tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)]

                if '기타' in cat_id_morphs:
                    df = tmp_df_filtered
                else:
                    if '긴팔' in cat_id_morphs:
                        cat_id_morphs += ['롱']
                    elif '반팔' in cat_id_morphs:
                        cat_id_morphs += ['숏']    
                    else:
                        cat_id_morphs = cat_id_morphs
                    df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))]
                    
                final_df = pd.concat([final_df, df], ignore_index=True)
            except:
                pass
        
    return final_df

def make_df_for_six_num_ids():
    morphs_list_without_subs, cat_list_without_subs = make_category_lists_without_subs()
    
    final_df = pd.DataFrame()
    for cat_id in list(sample_df['cat_id'].unique()):
        if cat_id in cat_list_without_subs:
            
            tmp_df = sample_df[sample_df['cat_id'] == cat_id].copy()
            cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))

            mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
            other_morphs = [morph for morph in morphs_list_without_subs if morph not in cat_id_morphs]
            opposite_keywords = '|'.join(other_morphs)

            tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)].copy()

            if '기타' in cat_id_morphs:
                df = tmp_df_filtered
            else:
                df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))].copy()
                
            final_df = pd.concat([final_df, df])
    
    return final_df


def concat_two_df(): 
    final_df1 = make_df_for_nine_nums_cat_ids()
    final_df2 = make_df_for_six_num_ids()
    df = pd.concat([final_df1, final_df2], ignore_index=True)
    return df # 대략 반개 줄어들음 

final_df = concat_two_df()
len(final_df)

/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/4021209625.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/4021209625.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_88473/4021209625.py:4: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')


681130

In [647]:
import matplotlib.pyplot as plt

plt.histfinal_df['cat_id'].value_counts()


310100999    9653
320090999    9599
320080999    9480
310040999    9441
320040999    9354
             ... 
310180060      74
310080050      57
320070300      41
320180         37
320140300      11
Name: cat_id, Length: 165, dtype: int64

In [645]:
cat_id = '310180060'
tmp_df = sample_df[sample_df['cat_id'] == cat_id]
cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))
mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
midcat_morphs = morphs_dict_with_subs[cat_id[:6]]

cat_id_morphs = [morph for morph in cat_id_morphs if morph in midcat_morphs]
other_morphs = [morph for morph in midcat_morphs if morph not in cat_id_morphs]
opposite_keywords = '|'.join(other_morphs)
print(opposite_keywords)

tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)]

if '기타' in cat_id_morphs:
    df = tmp_df_filtered
else:
    if '긴팔' in cat_id_morphs:
        cat_id_morphs += ['롱']
    elif '반팔' in cat_id_morphs:
        cat_id_morphs += ['숏']    
    else:
        cat_id_morphs = cat_id_morphs
    
    df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))]
df

자켓|세트|기타


,product_id,product_name,cat_id
51588,115982699,ZOOC 블랙 정장치마,310180060
51591,156832947,s 하이웨스트 수술달린 정장 치마 스커트,310180060
51594,212298212,여성 정장 치마 (겨울) 66 밤색 의류,310180060
51600,206999950,면접용 치마 팔아요 상태좋아요,310180060
51602,82093182,shesmiss 정장치마,310180060
...,...,...,...
1288253,100233030,정장 치마,310180060
1288254,222376329,77 새상품 스커트 정장 치마 미디 펜슬 여자 여성 에이치라인 오피스룩,310180060
1288259,195044601,(작은 사이즈로 수선) 로엠 정장 치마 정장 스커트,310180060
1288260,167731748,면접치마,310180060


In [642]:
cat_id = '400999'
tmp_df = sample_df[sample_df['cat_id'] == cat_id].copy()
cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))

mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
other_morphs = [morph for morph in morphs_list_without_subs if morph not in cat_id_morphs]
opposite_keywords = '|'.join(other_morphs)

tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)].copy()

if '기타' in cat_id_morphs:
    df = tmp_df_filtered
else:
    df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))].copy()

df

['기타', '액세서리']
티|맨투맨|테마|웨|스웨터|홈|언더웨어|니트|이벤트|업|점프수트|우산|후드|어|스니커즈|집|양산|가디건


,product_id,product_name,cat_id
153631,177804754,새제품 보라카이 열쇠고리 기념품 장식 소품,400999
153632,224799469,스투시 다이스 키링,400999
153633,162525089,고급키링 자동차 키링 가죽 차키 키홀더 열쇠고리 차량 스마트 가방 선물,400999
153635,217817283,에르메스 금장 켈리돌 한정판 백참 - U각인,400999
153636,214084071,E23/uroKo(우로코) 리메이크 프린지 키링[키홀더 백참],400999
...,...,...,...
1391167,207731152,루돌프 머리띠4,400999
1391168,181947973,담요 새상품 최저가 정가 16000,400999
1391169,149069579,레진으로 만든 별 머리핀,400999
1391171,164097143,전통매듭목걸이/목걸이/전통목걸이/쥬얼리/매듭,400999


In [ ]:
# -- 데이터 분포 확인하기 -- 